In [1]:
import tensorflow as tf
import cv2
from keras.datasets import mnist
from keras.layers import *
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import numpy as np
from matplotlib import pyplot as plt
import keras

In [2]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()

In [3]:
xtrain = (xtrain - 127.5)/127.5

In [4]:
totalepochs = 50
batchsize = 256
halfbatch = 128
numofbatch = int(xtrain.shape[0]/batchsize)
noisedim = 100
adam = Adam(learning_rate = 2e-4, beta_1 = 0.5)

In [5]:
generator = Sequential()
generator.add(Dense(units = 7 * 7 * 128, input_shape = (noisedim,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, (3,3), strides=(2,2), padding= "same"))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding= "same", activation="tanh"))
generator.compile(loss = keras.losses.binary_crossentropy, optimizer = adam)
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 7, 128)         0         
                                                                 
 batch_normalization (Batch  (None, 7, 7, 128)         512       
 Normalization)                                                  
                                                                 
 conv2d_transpose (Conv2DTr  (None, 14, 14, 64)        73792     
 anspose)                                                        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 64)        0

In [6]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding="same", input_shape= (28,28,1)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Conv2D(128, kernel_size=(3,3), strides=(2,2), padding="same"))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation="sigmoid"))
discriminator.compile(loss = keras.losses.binary_crossentropy, optimizer = adam)
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 100)               627300    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 100)              

In [7]:
discriminator.trainable = False
ganinput = Input(shape=noisedim, )
genimg = generator(ganinput)
ganoutput = discriminator(genimg)

In [8]:
model = Model(ganinput, ganoutput)
model.compile(loss=keras.losses.binary_crossentropy, optimizer= adam)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         708609    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 701897    
                                                                 
Total params: 1410506 (5.38 MB)
Trainable params: 708225 (2.70 MB)
Non-trainable params: 702281 (2.68 MB)
_________________________________________________________________


In [9]:
xtrain = xtrain.reshape(-1, 28, 28, 1)
xtrain.shape

(60000, 28, 28, 1)

In [10]:
def displayimages(samples = 25):
    noise = np.random.normal(0,1,size=(samples, noisedim))
    generatedimage = generator.predict(noise)
    plt.figure(figsize=(10, 10))
    for i in range(samples):
        plt.subplot(5,5,i+1)
        plt.imshow(generatedimage[i].reshape(28,28), cmap= "binary")
        plt.axis("off")
    plt.show()

In [11]:
dlosses = []
glosses = []
count = 0
for epoch in range(totalepochs):
    epochdlosses = 0.0
    epochglosses = 0.0
    for step in range(numofbatch):
        discriminator.trainable = True
        idx = np.random.randint(0,60000, halfbatch)
        realimage = xtrain[idx]
        noise = np.random.normal(0,1, size=(halfbatch,noisedim))
        fakeimage = generator.predict(noise)
        realy = np.ones((halfbatch, 1))*0.9
        fakey = np.zeros((halfbatch, 1))
        dlossreal = discriminator.train_on_batch(realimage, realy)
        dlossfake = discriminator.train_on_batch(fakeimage, fakey)
        dloss = 0.5 * dlossreal + 0.5 * dlossfake
        epochdlosses += dloss
        discriminator.trainable = False
        noise = np.random.normal(0,1,size=(batchsize, noisedim))
        groundtruthy = np.ones((batchsize, 1))
        gloss = model.train_on_batch(noise, groundtruthy)
        epochglosses += gloss
        print(f"epoch{epoch + 1}, discriminator losses{epochdlosses/numofbatch}, generator losses{epochglosses/numofbatch}")
        dlosses.append(epochdlosses/numofbatch)
        glosses.append(epochglosses/numofbatch)
        if(epoch + 1) % 10 == 0:
            generator.save("generator.h5")
        displayimages()

4/4 [==============================] - 0s 7ms/step


KeyError: in user code:

    File "C:\Users\Rishikeshs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Rishikeshs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Rishikeshs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Rishikeshs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training